<a href="https://colab.research.google.com/github/starirene9/DeepLearningAssignment/blob/main/conditionalGAN_CNN_K2024504_%EA%B5%AC%EB%B9%9B%EB%82%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import random
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data.sampler import SubsetRandomSampler

from torchsummary import summary

import torchvision.transforms as T
from torchvision.datasets import MNIST

from sklearn.manifold import TSNE

from matplotlib import pyplot as plt

In [44]:
def make_fig(z, targets, epoch):

    model = TSNE()
    transformed = model.fit_transform(z)

    plt.scatter(transformed[:, 0], transformed[:, 1], c=targets, alpha=.4, s=3**2)

    plt.savefig(fname=f'{epoch}.png', format='png')

In [45]:
LR = 0.0001
EPOCH = 200
BATCH_SIZE = 64

LATENT_DIM = 128

In [46]:
class SubsetSequentialSampler(torch.utils.data.Sampler):
    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)

train = MNIST('./', train=True, download=True,
              transform=T.Compose([T.ToTensor(),
                                   T.Normalize((0.5,), (0.5,)),])
             )
test = MNIST('./', train=False, download=True, transform=T.ToTensor())

train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, num_workers=2, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE)

In [47]:
class Residual(nn.Module):
    def __init__(self, channel):
        super(Residual, self).__init__()

        self.conv1 = nn.Conv2d(channel, channel, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(channel, channel, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(channel)
        self.bn2 = nn.BatchNorm2d(channel)

        self.leaky = nn.LeakyReLU(inplace=False)

    def forward(self, x):
        out = self.leaky(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = out + x
        return self.leaky(out)


class Pooling(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(Pooling, self).__init__()

        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1)

        self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)

        self.leaky = nn.LeakyReLU(inplace=False)

    def forward(self, x):
        out_ = self.leaky(self.bn1(self.conv1(x)))
        return out_

In [48]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_emb = nn.Embedding(10, 28*28)  # 🔹 레이블을 이미지 크기로 임베딩

        self.block_1 = nn.Sequential(Residual(2), Pooling(2, 8))  # 14
        self.block_2 = nn.Sequential(Residual(8), Pooling(8, 16))  # 7

        self.fc = nn.Linear(16, 1)

        self.GAP = nn.AdaptiveAvgPool2d(1)

        self.leaky = nn.LeakyReLU(inplace=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, y):
        y = self.label_emb(y).view(-1, 1, 28, 28)  # 🔹 임베딩 결과를 이미지처럼 reshape
        x = torch.cat((x, y), dim=1)  # 🔹 이미지와 라벨을 채널 방향으로 합침

        out = self.block_1(x)
        out = self.block_2(out)

        out = self.GAP(out)

        out = out.view(out.size(0), -1)

        out = self.fc(out)

        return self.sigmoid(out)

In [49]:
class Transpose(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(Transpose, self).__init__()

        self.conv_trans = nn.ConvTranspose2d(in_channels=in_channel,
                                             out_channels=out_channel,
                                             kernel_size=4, stride=2, padding=1) # 사이즈 2배 upscaling
        self.conv1 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1)

        self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)

        self.leaky = nn.LeakyReLU(inplace=False)

    def forward(self, x):
        out_ = self.leaky(self.bn1(self.conv_trans(x)))
        out = self.bn2(self.conv1(out_))
        out = out + out_
        return out

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(10, 10)

        self.fc = nn.Linear(LATENT_DIM + 10, 7*7*16)

        self.bn = nn.BatchNorm1d(7*7*16)

        self.transpose_1 = Transpose(16, 8)
        self.transpose_2 = Transpose(8, 1)

        self.leaky = nn.LeakyReLU(inplace=False)
        self.tanh = nn.Tanh()

    def forward(self, x, y):
        y = self.label_emb(y)  # 🔹 one-hot과 유사한 역할 : label을 학습 가능한 continuous vector로 표현 (더 유연함)
        x = torch.cat((x, y), dim=1)  # 🔹 noise + label 결합
        out = self.leaky(self.bn(self.fc(x)))

        out = out.view(out.size(0), 16, 7, 7)

        out = self.leaky(self.transpose_1(out))
        out = self.transpose_2(out)

        return self.tanh(out)

In [50]:
# ✅ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

disc = Discriminator().to(device)
gen = Generator().to(device)

g_optim = torch.optim.Adam(gen.parameters(), lr=2e-4)
d_optim = torch.optim.Adam(disc.parameters(), lr=5e-5)

criterion = nn.BCELoss().cuda()

In [51]:
def train_discriminator(criterion, optimizer, real_data, real_labels, fake_data, fake_labels):
    n = real_data.size(0)

    optimizer.zero_grad()

    prediction_real = disc(real_data, real_labels)
    loss_real = criterion(prediction_real, torch.ones(n, 1).to(device))
    loss_real.backward()

    prediction_fake = disc(fake_data.detach(), fake_labels)
    loss_fake = criterion(prediction_fake, torch.zeros(n, 1).to(device))
    loss_fake.backward()

    optimizer.step()

    return loss_real + loss_fake

def train_generator(criterion, optimizer, fake_data, labels):
    n = fake_data.size(0)
    optimizer.zero_grad()

    prediction = disc(fake_data, labels)
    loss = criterion(prediction, torch.ones(n, 1).to(device))

    loss.backward()
    optimizer.step()

    return loss


In [ ]:
for epoch in range(EPOCH):
    gen.train()
    disc.train()
    for img, target in tqdm(train_loader, leave=False, total=len(train_loader)):
        img = img.to(device)
        target = target.to(device)

        n = len(img)

        noise = torch.randn(n, LATENT_DIM).to(device)
        fake_data = gen(noise, target)
        disc_loss = train_discriminator(criterion, d_optim, img, target, fake_data, target)

        noise = torch.randn(n, LATENT_DIM).to(device)
        fake_data = gen(noise, target)
        gen_loss = train_generator(criterion, g_optim, fake_data, target)

    if not epoch % 10:
        print(f"Epoch {epoch} | D Loss: {disc_loss:.4f} | G Loss: {gen_loss:.4f}")

        gen.eval()  # 🔸 평가 모드로 변경 (BatchNorm 문제 방지)
        with torch.no_grad():  # 🔸 그래디언트 비활성화
            z = torch.randn(1, LATENT_DIM).to(device)
            label = torch.tensor([7]).to(device)
            generated_img = gen(z, label)[0]
            generated_img = (generated_img + 1) / 2  # 🔸 Tanh 정규화 해제: [-1, 1] → [0, 1]
            display(T.functional.to_pil_image(generated_img))

Epoch 0 | D Loss: 1.3347 | G Loss: 0.7239


Epoch 10 | D Loss: 1.3609 | G Loss: 0.7475


Epoch 20 | D Loss: 0.6647 | G Loss: 1.6886


 18%|█▊        | 165/938 [00:10<00:50, 15.25it/s]

상태	해석
D Loss ≫ G Loss	Generator가 못 속임 → Generator 더 훈련 필요
G Loss ≫ D Loss	Discriminator가 너무 잘 구분함 → Generator 손해
D Loss ≈ G Loss (모두 적당히 높음)	서로 경쟁 중 → 가장 이상적인 상태!

In [ ]:
from IPython.display import display
display(T.functional.to_pil_image((gen(torch.randn(1, LATENT_DIM).to(device), torch.tensor([7]).to(device))[0] + 1) / 2))